In [ ]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import metrics
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import ConvNeXtBase
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
# Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')
print(train_df.shape[0])

# Generate file paths correctly
train_df['file_path'] = train_df.apply(
    lambda row: f"/kaggle/input/bttai-ajl-2025/train/train/{row['label']}/{row['md5hash']}.jpg", axis=1
)
test_df['file_path'] = test_df['md5hash'].apply(
    lambda x: f"/kaggle/input/bttai-ajl-2025/test/test/{x}.jpg"
)

# Data Preprocessing

# Remove invalid rows
train_df = train_df[(train_df['fitzpatrick_scale'] > 0) & (train_df['label'].notna())]
print(train_df.shape[0])

train_df = train_df[train_df['file_path'].apply(os.path.exists)]
print(train_df.shape[0])

test_df = test_df[test_df['file_path'].apply(os.path.exists)]
print(test_df.shape[0])
print()


# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])


# Splitting dataset into training and validation datasets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['encoded_label'])

# Define image data generators for training and testing
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.9, 1.1],
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=512,
    class_mode='raw',
    shuffle = True
)


val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=512,
    class_mode='raw',
    shuffle=False
    
)


test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    target_size=(224, 224),
    batch_size= 512,
    class_mode=None,
    shuffle=False
    
)


In [ ]:
# Common Model Parameters
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df['encoded_label']),
    y=train_df['encoded_label']
)
class_weights_dict = dict(enumerate(class_weights))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Initialize the ReduceLROnPlateau callback
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.5,  # Factor to reduce the learning rate
                                 patience=3,  # Number of epochs to wait before reducing
                                 min_lr=1e-6)  # Minimum learning rate

checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

In [ ]:
# Load ConvNeXtTiny with pre-trained weights
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model


# Build the model
model_eff = models.Sequential([
    base_model,  # Base model (ConvNeXtTiny)
    layers.GlobalAveragePooling2D(),  # Pooling layer to reduce spatial dimensions
    layers.BatchNormalization(),  # Batch normalization to stabilize training
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # First dense layer with more units
    layers.Dropout(0.4),  # Increased dropout rate to prevent overfitting
    layers.BatchNormalization(),  # Another batch normalization layer
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # Second dense layer
    layers.Dropout(0.4),  # Dropout layer for regularization
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # Third dense layer
    layers.Dropout(0.3),  # Dropout layer
    layers.Dense(21, activation='softmax')  # Output layer with 21 classes (adjust accordingly)
])

# Compile the model
model_eff.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_eff.fit(train_generator, epochs=40, validation_data=val_generator, callbacks=[lr_reduction, checkpoint])

In [ ]:
# Load ConvNeXtTiny with pre-trained weights
base_model = ConvNeXtBase(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model


# Build the model
model_conv = models.Sequential([
    base_model,  # Base model (ConvNeXtTiny)
    layers.GlobalAveragePooling2D(),  # Pooling layer to reduce spatial dimensions
    layers.BatchNormalization(),  # Batch normalization to stabilize training
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # First dense layer with more units
    layers.Dropout(0.4),  # Increased dropout rate to prevent overfitting
    layers.BatchNormalization(),  # Another batch normalization layer
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # Second dense layer
    layers.Dropout(0.4),  # Dropout layer for regularization
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # Third dense layer
    layers.Dropout(0.3),  # Dropout layer
    layers.Dense(21, activation='softmax')  # Output layer with 21 classes (adjust accordingly)
])

# Compile the model
model_conv.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_conv.fit(train_generator, epochs=40, validation_data=val_generator, callbacks=[lr_reduction, checkpoint])

In [ ]:
# Generate predictions (probabilities)
y_prob_conv = model_conv.predict(val_generator)
y_prob_eff = model_eff.predict(val_generator)
y_true = val_df['encoded_label'].values

# Average the probabilities from both models
y_prob_avg = (y_prob_conv + y_prob_eff) / 2

# Convert averaged probabilities to predicted class labels
y_pred_avg = np.argmax(y_prob_avg, axis=1)

# Calculate the F1 Score for the averaged predictions
f1_avg = f1_score(y_true, y_pred_avg, average='weighted')
print("Average F1 Score:", f1_avg)


In [ ]:
# SUBMISSION.CSV

# Generate predictions
y_prob_conv = model_conv.predict(test_generator)  # Probabilities from ConvNet
y_prob_eff = model_eff.predict(test_generator)  # Probabilities from EfficientNet

# Average the probabilities
y_prob= (y_pred_conv + y_pred_eff) / 2

# Get the predicted class by taking the argmax
y_pred = np.argmax(y_prob, axis=1)

# Save submission
test_df['label'] = label_encoder.inverse_transform(y_pred)
test_df[['md5hash', 'label']].to_csv('/kaggle/working/submission.csv', index=False)